In [74]:
before = pd.read_csv("auto_baseline.csv")
after = pd.read_csv("mod.csv")
compare_out_file = mode + "_mod_compare.csv"

In [75]:
df = pd.merge(before, 
              after, 
              on=['Origin_TAZID', 'Destination_TAZID'], 
              how="inner", 
              suffixes=("_before", "_after")
)

In [76]:
df['diff_hh'] = df['weighted_hh_after'] - df['weighted_hh_before']
df['diff_jobs'] = df['weighted_jobs_after'] - df['weighted_jobs_before']
df['diff_ato'] = df['ato_after'] - df['ato_before']

In [77]:
#df.groupby('Origin_CO_TAZID_before')['diff_ato'].sum()

In [78]:
df.head()

,Unnamed: 0_before,Origin_TAZID,Destination_TAZID,Total_Time_before,survey_weight_before,weighted_jobs_before,weighted_hh_before,ato_before,Unnamed: 0_after,Total_Time_after,survey_weight_after,weighted_jobs_after,weighted_hh_after,ato_after,diff_hh,diff_jobs,diff_ato
0,0,351496,351496,0.000000,1.000,325.0,279.0,604.0,0,0.000000,1.000,325.0,279.0,604.0,0.0,0.0,0.0
1,1,351490,351496,2.581873,1.000,325.0,279.0,604.0,1,2.581873,1.000,325.0,279.0,604.0,0.0,0.0,0.0
2,2,351488,351496,3.876465,0.981,319.0,274.0,593.0,2,3.876465,0.981,319.0,274.0,593.0,0.0,0.0,0.0
3,3,351494,351496,3.772201,0.985,320.0,275.0,595.0,3,3.772201,0.985,320.0,275.0,595.0,0.0,0.0,0.0
4,4,351497,351496,2.417536,1.000,325.0,279.0,604.0,4,2.417536,1.000,325.0,279.0,604.0,0.0,0.0,0.0


In [79]:
# assert df['diff_ato'].min() >= 0

In [80]:
summary = df.groupby('Origin_TAZID').agg(
    jobs=pd.NamedAgg(column='diff_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='diff_hh', aggfunc=sum),
    ato=pd.NamedAgg(column='diff_ato', aggfunc=sum))

In [81]:
summary['ato'] = summary['jobs'] + summary['hh'] * 1.8

In [82]:
summary.to_csv(compare_out_file)

In [83]:
df.agg(
    jobs=pd.NamedAgg(column='diff_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='diff_hh', aggfunc=sum),
    ato=pd.NamedAgg(column='diff_ato', aggfunc=sum))

,diff_jobs,diff_hh,diff_ato
jobs,136189.0,NaN,NaN
hh,NaN,51413.0,NaN
ato,NaN,NaN,187602.0


## Visualize Output

In [84]:
arcpy.management.MakeFeatureLayer(
    os.path.join(base_path, r"shp\taz_wfrc.gdb\ATO"), 
    "taz_compare"
)

<Result 'taz_compare'>

In [85]:
arcpy.conversion.TableToTable(
    os.path.join(base_path, compare_out_file), 
    base_gdb , 
    "mod_compare"
)

<Result 'C:\\wfrc\\ato\\baseline.gdb\\mod_compare'>

In [87]:
arcpy.management.AddJoin(
    "taz_compare", 
    "CO_TAZID", 
    "mod_compare", 
    "Origin_TAZID", 
    "KEEP_ALL"
)

<Result 'taz_compare'>

Now, manually apply symbology to taz_compare layer (I experimented with creating a layer style and applying programatically), but found it easier just to manually update)
